In [ ]:
### Initial directory must contain the following files: HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii 
### (downloaded from HCP website), cope1.dtseries.nii (one random cope file from the HCP project), 
### dscalar_names.txt (text file with eight columns including the numbers from 1 to 8), 
### Cerebellum-MNIfnirt-maxprob-thr25.nii (cerebellum atlas, will be used to isolate cerebellum), 
### mapalign folder (add it to initial folder), hcp.tmp.lh.dscalar.nii and 
### hcp.tmp.rh.dscalar.nii (from the HCP project) and the mapalign folder.
### These are included in folder "files_for_calculating_gradients_group", with the exception of 
### HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii.
### Also, workbench command (wb_command), Matlab and SUIT toolbox must work in current machine

In [ ]:
import subprocess
import numpy as np
import nibabel as nib
from sklearn.metrics import pairwise_distances
import sys
sys.path.append("./mapalign")
from mapalign import embed
from PIL import Image

In [ ]:
cd /files_for_calculating_gradients_group

In [ ]:
### Restrict dconn file so that it includes cerebellum only
subprocess.check_output("wb_command -cifti-restrict-dense-map HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii COLUMN HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr_cerebellumonly_step1.dconn.nii -vol-roi Cerebellum-MNIfnirt-maxprob-thr25.nii", shell=True);
subprocess.check_output("wb_command -cifti-restrict-dense-map HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr_cerebellumonly_step1.dconn.nii ROW HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr_cerebellumonly_step2.dconn.nii -vol-roi Cerebellum-MNIfnirt-maxprob-thr25.nii", shell=True);

In [ ]:
### Generate dscalar file which includes the cerebellum only and has 8 maps (which is the number of gradients that the following steps will want to generate):
subprocess.check_output("wb_command -cifti-restrict-dense-map cope1.dtseries.nii COLUMN cope1_cerebellumonly.dtseries.nii -vol-roi Cerebellum-MNIfnirt-maxprob-thr25.nii", shell=True);
subprocess.check_output("wb_command -cifti-merge mergedfile8_cerebellumonly.dtseries.nii -cifti cope1_cerebellumonly.dtseries.nii -cifti cope1_cerebellumonly.dtseries.nii -cifti cope1_cerebellumonly.dtseries.nii -cifti cope1_cerebellumonly.dtseries.nii -cifti cope1_cerebellumonly.dtseries.nii -cifti cope1_cerebellumonly.dtseries.nii -cifti cope1_cerebellumonly.dtseries.nii -cifti cope1_cerebellumonly.dtseries.nii", shell=True);
subprocess.check_output("wb_command -cifti-change-mapping mergedfile8_cerebellumonly.dtseries.nii ROW mergedfile8_cerebellumonly.dscalar.nii -scalar -name-file dscalar_names.txt", shell=True);

In [ ]:
### Generate .dscalar file containing gradients
dcon = np.tanh(nib.load('HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr_cerebellumonly_step2.dconn.nii').get_data())
N = dcon.shape[0]
perc = np.array([np.percentile(x, 90) for x in dcon])
for i in range(dcon.shape[0]):
    print "Row %d" % i
    dcon[i, dcon[i,:] < perc[i]] = 0
dcon[dcon < 0] = 0
aff = 1 - pairwise_distances(dcon, metric = 'cosine')
np.save('cosine_affinity_cerebellumonly.npy', aff)
aff = np.load('cosine_affinity_cerebellumonly.npy')
emb, res = embed.compute_diffusion_map(aff, alpha = 0.5)
np.save('embedding_dense_cerebellumonly_emb.npy', emb)
np.save('embedding_dense_cerebellumonly_res.npy', res)
res = np.load('embedding_dense_cerebellumonly_res.npy').item()
a = [res['vectors'][:,i]/ res['vectors'][:,0] for i in range(134)]
emb = np.array(a)[1:,:].T
len(emb)
res = nib.load('hcp.tmp.lh.dscalar.nii').get_data()
cortL = np.squeeze(np.array(np.where(res != 0) [0], dtype=np.int32))
res = nib.load('hcp.tmp.rh.dscalar.nii').get_data()
cortR = np.squeeze(np.array(np.where(res != 0) [0], dtype=np.int32))
corLen = len(cortL) + len(cortR)
del res
emb = np.load('embedding_dense_cerebellumonly_emb.npy')
emb.shape
tmp = nib.load('mergedfile8_cerebellumonly.dscalar.nii')
tmp_cifti = nib.cifti2.load('mergedfile8_cerebellumonly.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_cerebellumonly.dscalar.nii')

In [ ]:
### Separate each gradient in a different dscalar file
import nibabel as nib
import numpy as np
res = nib.load('hcp.tmp.lh.dscalar.nii').get_data()
cortL = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
res = nib.load('hcp.tmp.rh.dscalar.nii').get_data()
cortR = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
cortLen = len(cortL) + len(cortR)
del res
emb = np.load('embedding_dense_cerebellumonly_emb.npy')
tmp = nib.load('cope1_cerebellumonly.dscalar.nii')
tmp_cifti = nib.cifti2.load('cope1_cerebellumonly.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
emb_temporary = emb[:,0]
emb_temporary = emb_temporary.T
emb_temporary.shape = (1, 18142)
img = nib.cifti2.Cifti2Image(emb_temporary, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_cerebellumonly_gradient1.dscalar.nii')
emb_temporary = emb[:,1]
emb_temporary = emb_temporary.T
emb_temporary.shape = (1, 18142)
img = nib.cifti2.Cifti2Image(emb_temporary, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_cerebellumonly_gradient2.dscalar.nii')
emb_temporary = emb[:,2]
emb_temporary = emb_temporary.T
emb_temporary.shape = (1, 18142)
img = nib.cifti2.Cifti2Image(emb_temporary, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_cerebellumonly_gradient3.dscalar.nii')
emb_temporary = emb[:,3]
emb_temporary = emb_temporary.T
emb_temporary.shape = (1, 18142)
img = nib.cifti2.Cifti2Image(emb_temporary, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_cerebellumonly_gradient4.dscalar.nii')
emb_temporary = emb[:,4]
emb_temporary = emb_temporary.T
emb_temporary.shape = (1, 18142)
img = nib.cifti2.Cifti2Image(emb_temporary, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_cerebellumonly_gradient5.dscalar.nii')
emb_temporary = emb[:,5]
emb_temporary = emb_temporary.T
emb_temporary.shape = (1, 18142)
img = nib.cifti2.Cifti2Image(emb_temporary, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_cerebellumonly_gradient6.dscalar.nii')
emb_temporary = emb[:,6]
emb_temporary = emb_temporary.T
emb_temporary.shape = (1, 18142)
img = nib.cifti2.Cifti2Image(emb_temporary, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_cerebellumonly_gradient7.dscalar.nii')
emb_temporary = emb[:,7]
emb_temporary = emb_temporary.T
emb_temporary.shape = (1, 18142)
img = nib.cifti2.Cifti2Image(emb_temporary, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_cerebellumonly_gradient8.dscalar.nii')

In [ ]:
### Transform from dscalar to nifti format
import subprocess
subprocess.check_output("wb_command -cifti-separate \
result_cerebellumonly_gradient1.dscalar.nii COLUMN -volume-all  \
result_cerebellumonly_gradient1_nifti.nii", shell=True);
subprocess.check_output("wb_command -cifti-separate \
result_cerebellumonly_gradient2.dscalar.nii COLUMN -volume-all \
result_cerebellumonly_gradient2_nifti.nii", shell=True);
subprocess.check_output("wb_command -cifti-separate \
result_cerebellumonly_gradient3.dscalar.nii COLUMN -volume-all \
result_cerebellumonly_gradient3_nifti.nii", shell=True);
subprocess.check_output("wb_command -cifti-separate \
result_cerebellumonly_gradient4.dscalar.nii COLUMN -volume-all \
result_cerebellumonly_gradient4_nifti.nii", shell=True);
subprocess.check_output("wb_command -cifti-separate \
result_cerebellumonly_gradient5.dscalar.nii COLUMN -volume-all \
result_cerebellumonly_gradient5_nifti.nii", shell=True);
subprocess.check_output("wb_command -cifti-separate \
result_cerebellumonly_gradient6.dscalar.nii COLUMN -volume-all \
result_cerebellumonly_gradient6_nifti.nii", shell=True);
subprocess.check_output("wb_command -cifti-separate \
result_cerebellumonly_gradient7.dscalar.nii COLUMN -volume-all \
result_cerebellumonly_gradient7_nifti.nii", shell=True);
subprocess.check_output("wb_command -cifti-separate \
result_cerebellumonly_gradient8.dscalar.nii COLUMN -volume-all \
result_cerebellumonly_gradient8_nifti.nii", shell=True);

In [ ]:
### Show gradients in cerebellum flatmap
subprocess.check_output('scp result_cerebellumonly_gradient1_nifti.nii \
image_nifti.nii', shell=True);
subprocess.check_output('bash call_matlab.sh', shell=True);
Image.open('image.jpg').show()

subprocess.check_output('scp result_cerebellumonly_gradient2_nifti.nii \
image_nifti.nii', shell=True);
subprocess.check_output('bash call_matlab.sh', shell=True);
Image.open('image.jpg').show()

subprocess.check_output('scp result_cerebellumonly_gradient3_nifti.nii \
image_nifti.nii', shell=True);
subprocess.check_output('bash call_matlab.sh', shell=True);
Image.open('image.jpg').show()

subprocess.check_output('scp result_cerebellumonly_gradient4_nifti.nii \
image_nifti.nii', shell=True);
subprocess.check_output('bash call_matlab.sh', shell=True);
Image.open('image.jpg').show()

subprocess.check_output('scp result_cerebellumonly_gradient5_nifti.nii \
image_nifti.nii', shell=True);
subprocess.check_output('bash call_matlab.sh', shell=True);
Image.open('image.jpg').show()

subprocess.check_output('scp result_cerebellumonly_gradient6_nifti.nii \
image_nifti.nii', shell=True);
subprocess.check_output('bash call_matlab.sh', shell=True);
Image.open('image.jpg').show()

subprocess.check_output('scp result_cerebellumonly_gradient7_nifti.nii \
image_nifti.nii', shell=True);
subprocess.check_output('bash call_matlab.sh', shell=True);
Image.open('image.jpg').show()

subprocess.check_output('scp result_cerebellumonly_gradient8_nifti.nii \
image_nifti.nii', shell=True);
subprocess.check_output('bash call_matlab.sh', shell=True);
Image.open('image.jpg').show()

In [ ]:
### call_matlab.sh contains the following:
# matlab -nodesktop -nosplash -r "run codetomovetoSUITspaceandplotflatmap.m"

### codetomovetoSUITspaceandplotflatmap.m contains the following:
#addpath /spm12
#addpath /spm12/compat
#addpath /spm12/toolbox/DARTEL
#addpath /spm12/toolbox/suit
#job.subj.affineTr = {'/Affine_MNI152_T1_2mm_seg1.mat'};
#job.subj.flowfield = {'/u_a_MNI152_T1_2mm_seg1.nii,1'};
#job.subj.resample = {'image_nifti.nii,1'};
#job.subj.mask = {'/c_MNI152_T1_2mm_pcereb.nii,1'};
#job.interp = 0;
#job.prefix = 'wc';
#
#suit_reslice_dartel(job)
#
#figure
#Data = suit_map2surf('/wcimage_nifti.nii','space','SUIT', 'stats',@mode)
#suit_plotflatmap(Data,'cmap',jet)
#savefig('figure')
#fig = openfig('figure.fig');
#filename = 'image.jpg';
#saveas(fig, filename)
#clearvars

### The files Affine_MNI152_T1_2mm_seg1.mat, u_a_MNI152_T1_2mm_seg1.nii and c_MNI152_T1_2mm_pcereb.nii
### are generated using the "isolate" and "normalize using Dartel" of the SUIT toolbox (http://www.diedrichsenlab.org/imaging/suit_function.htm)
### MNI152_T1_2mm is the structural space used in the Human Connectome Project.